## 1. Instalando o Faiss

In [1]:
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org faiss-cpu
#!pip install faiss-gpu

# CPU-only version
#$ conda install -c pytorch faiss-cpu=1.8.0 --verbose -y

# GPU(+CPU) version
#$ conda install -c pytorch -c nvidia faiss-gpu=1.8.0

# GPU(+CPU) version with NVIDIA RAFT
#$ conda install -c pytorch -c nvidia -c rapidsai -c conda-forge faiss-gpu-raft=1.8.0

## 2. Criando os vetores com os dados

In [26]:
import numpy as np

# d é o tamanho dos vetores (embeddings)
d = 1024                           # dimension

# nb é a quantidade de vetores indexados no banco de dados
nb = 16045                      # database size

# nq é o número de queries
nq = 150                       # nb of queries

np.random.seed(1234)             # make reproducible

# xb é a matriz que contem todos os vetores que serão indexados no banco. Tamanho nb x d.
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.

#id_list é a lista de ids dos vetores em xb
id_list = [x for x in range(nb, 0, -1)]

# xq é a matriz que contém todos os vetores de queries. Tamanho nq x d
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [23]:
xb

array([[ 0.19151945,  0.62210876,  0.43772775, ...,  0.60382867,
         0.18178934,  0.5190297 ],
       [ 0.71115327,  0.9249804 ,  0.52913165, ...,  0.72927713,
         0.4249684 ,  0.72562623],
       [ 0.5900075 ,  0.13707106,  0.5647468 , ...,  0.89168775,
         0.5799966 ,  0.72705346],
       ...,
       [16.464464  ,  0.2699034 ,  0.4109476 , ...,  0.44040364,
         0.8484792 ,  0.49044716],
       [16.318378  ,  0.6963395 ,  0.28584644, ...,  0.62822104,
         0.45907068,  0.3205342 ],
       [16.701004  ,  0.55168337,  0.59184545, ...,  0.86712843,
         0.2974102 ,  0.06862216]], dtype=float32)

## 3. Construindo um índice e adicionando os vetores a ele

In [27]:
import faiss                   # make faiss available

#Cria índice com versão mais simples que apenas realiza uma busca exaustiva de distância L2
index = faiss.IndexIDMap(faiss.IndexFlatL2(d))   # build the index
#index = faiss.IndexFlatL2(d)

# Os índices precisam saber qual é a dimensionalidade dos vetores e a maioria requer uma fase de treinamento
# Para o IndexFlatL2, podemos pular essa operação

# Verifica se o índice já está treinado
print(index.is_trained)

# Adiciona os vetores ao índice do banco de dados
index.add_with_ids(xb, id_list)                 # add vectors to the index
#index.add(xb)

# Verifica o número de vetores indexados
print(index.ntotal)

True
16045


## 4. Buscando

In [28]:
# k é o número de vetores que serão retornados na busca
k = 4                          # we want to see 4 nearest neighbors

# teste de sanidade do banco
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

# busca propriamente dita
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

# I é uma matriz inteira de tamanho nq x k, onde a linha i contém os IDs dos k vizinhos do vetor de consulta i,
# ordenados por distância crescente

# D é uma matriz de ponto flutuante nq x k com as distâncias quadráticas correspondentes

[[16045 15795 15997 15257]
 [16044 15691 16007 15679]
 [16043 15571 15000 15933]
 [16042 15538 16004 15540]
 [16041 14399 14555 14885]]
[[  0.      156.57214 156.74513 158.37045]
 [  0.      156.001   156.1425  156.37476]
 [  0.      154.04337 154.29654 154.31151]
 [  0.      149.58832 150.87465 152.00418]
 [  0.      152.05606 152.75455 152.84671]]
[[14533 15024 15227 15663]
 [14800 15362 15194 15927]
 [15814 15769 15684 16030]
 [14596 15434 15191 15584]
 [15024 15396 15730 15882]]
[[15869 15649 15760 14934]
 [15742 15814 15778 15997]
 [15776 15210 15257 14485]
 [14383 15540 14914 15051]
 [14904 14868 14790 13983]]


Para realizar uma pesquisa semântica com termos de pesquisa em uma base de jurisprudência do TCU, você provavelmente deseja utilizar um tipo de índice que seja eficiente para calcular similaridades entre os vetores que representam os documentos da jurisprudência e os vetores que representam os termos de pesquisa.

O Faiss oferece vários tipos de índices que podem ser adequados para essa tarefa, dependendo das características específicas do seu conjunto de dados e das necessidades da sua aplicação. Aqui estão algumas sugestões:

IndexFlatL2: Este é o tipo mais simples de índice e pode ser uma escolha adequada se você estiver lidando com um conjunto de dados relativamente pequeno e a eficiência não for uma preocupação principal. No entanto, se você estiver trabalhando com uma base de jurisprudência grande, o IndexFlatL2 pode não ser a melhor escolha devido à sua complexidade de busca linear.

IndexIVFFlat: Este é um tipo de índice que divide o espaço de vetores em células usando quantização vetorial e usa a busca aproximada dentro de cada célula para acelerar a busca. Pode ser uma boa opção se você estiver lidando com um conjunto de dados grande e quiser equilibrar eficiência e precisão.

IndexHNSWFlat: Este é um tipo de índice que constrói uma estrutura de vizinhança hierárquica esparsa (HNSW) para acelerar a busca de vizinhos mais próximos. Pode ser uma escolha adequada se a eficiência de busca for uma prioridade e você estiver disposto a sacrificar um pouco de precisão em troca de velocidade.

IndexIDMap + IndexLSH: Esta é uma combinação de índices onde o IndexIDMap armazena os vetores e seus IDs correspondentes, enquanto o IndexLSH é usado para acelerar a busca aproximada. Pode ser uma boa opção se você estiver lidando com um conjunto de dados grande e quiser uma solução simples e eficiente.

IndexPQ: Este é um tipo de índice que usa quantização de produto para acelerar a busca aproximada. Pode ser uma escolha adequada se você estiver lidando com um conjunto de dados grande e quiser uma solução eficiente em termos de memória e tempo de busca.

A escolha do tipo de índice depende de vários fatores, como o tamanho do conjunto de dados, os requisitos de eficiência de busca e a precisão desejada. Você pode experimentar diferentes tipos de índices e ajustar os parâmetros conforme necessário para encontrar a melhor solução para o seu caso de uso específico.